# Bibliotecas

In [1]:
import numpy as np
from scipy.io import wavfile
from scipy.io.wavfile import write

# Carregar áudio

In [11]:
# Ler arquivo wav
rate, data = wavfile.read('audios/NubiaCantaDalva.wav')

print(data.shape)
print(data.dtype)
rate

(1042432,)
int16


44100

&#8594; Rate = Taxa de amostragem do áudio, percebe-se que o resultado é de 44100 Hz;

&#8594; Data = Amplitudes do sinal amostrado.

Ao realizar "print(data.shape)", percebe-se que retorna um valor "(1042432,)", ou seja, se trata de um sinal MONO. Caso fosse estéreo, teria outro componente após a vírgula, representando um array bidimensional (Lado esquerdo e lado direito).

In [3]:
print(data.min(), data.max())

-32294 31545


# Normalização do áudio

&#8594; Esta normalização é necessária para facilitar o processamento do sinal e evitar Overflow, passando o sinal de int16 para um float que varia de -1 a 1.

In [4]:
data = data / np.max(np.abs(data))  # Normaliza os dados para o intervalo [-1, 1]
data

array([0., 0., 0., ..., 0., 0., 0.])

In [5]:
freq_data = np.fft.fft(data)
freq_data

array([2010.53966681 +0.j        , -149.47266855-19.22293337j,
       -184.64522435 +9.17095819j, ..., -146.47848095-26.10877145j,
       -184.64522435 -9.17095819j, -149.47266855+19.22293337j])

"freq_data" é um array complexo com os componentes de amplitudes e fases das frequências do áudio. 

In [6]:
frequencias = np.fft.fftfreq(len(data), d=1/rate)
frequencias

array([ 0.        ,  0.04230492,  0.08460984, ..., -0.12691475,
       -0.08460984, -0.04230492])

"frequencias" é uma varíavel que gera as frequências do áudio que correspondem ao freq_data. Definido pelas frequências da Transformada Rápida Discreta de Fourier (DFT) de data e usando o intervalo de amostragem como o inverso da taxa de amostragem.

# Filtros

In [7]:
# Frequências de corte
corte_baixo = 300 # 300hz
corte_alto = 2000 # 2Khz

Frequências de corte definidas, uma como sendo a frequência de corte para baixas e outra para altas.

In [8]:
def filtro_passa_baixas(frequencias, corte_baixo):
    return np.abs(frequencias) < corte_baixo

def filtro_passa_altas(frequencias, corte_alto):
    return np.abs(frequencias) > corte_alto

def filtro_passa_banda(frequencias, corte_baixo, corte_alto):
    return (np.abs(frequencias) > corte_baixo) & (np.abs(frequencias) < corte_alto)

def filtro_rejeita_banda(frequencias, corte_baixo, corte_alto):
    return (np.abs(frequencias) < corte_baixo) | (np.abs(frequencias) > corte_alto)

__Funções__
* Passa-baixas: Retorna todas as frequências abaixo do corte baixo;
* Passa-altas: Retorna todas as frequências acima do corte alto;
* Passa-banda: Retorna todas as frequências entre o corte baixo e o alto;
* Rejeita-banda: Retorna todas as frequências que não estão entre o corte baixo e o alto.

In [9]:
passa_baixas = filtro_passa_baixas(frequencias, corte_baixo)
passa_altas = filtro_passa_altas(frequencias, corte_alto)
passa_banda = filtro_passa_banda(frequencias, corte_baixo, corte_alto)
rejeita_faixa = filtro_rejeita_banda(frequencias, corte_baixo, corte_alto)

passa_baixas_filtrado = np.fft.ifft(freq_data * passa_baixas).real
passa_altas_filtrado = np.fft.ifft(freq_data * passa_altas).real
passa_banda_filtrado = np.fft.ifft(freq_data * passa_banda).real
rejeita_banda_filtrado = np.fft.ifft(freq_data * rejeita_faixa).real

As funções são executadas e então, as frequências retornada dos filtros são multiplicadas com as amplitudes e fases do áudio, aceitando somente números reais resultantes da Transformada Rápida Inversa de Fourier (IFFT).

In [10]:
wavfile.write('audios/NubiaCantaDalvaPassaBaixas.wav', rate, (passa_baixas_filtrado * 32767).astype(np.int16))
wavfile.write('audios/NubiaCantaDalvaPassaAltas.wav', rate, (passa_altas_filtrado * 32767).astype(np.int16))
wavfile.write('audios/NubiaCantaDalvaPassaBanda.wav', rate, (passa_banda_filtrado * 32767).astype(np.int16))
wavfile.write('audios/NubiaCantaDalvaRejeitaBanda.wav', rate, (rejeita_banda_filtrado * 32767).astype(np.int16))

Salva os novos áudios filtrados